# Convert a tsv file to csv

Covert a file from tsv format to csv format.  This is a demo of how to do this with 
tensorflow.

In order to import into BigQuery you need a csv file.  It appears that the gui import for 
BigQuery does not support tsv files.  Not sure about command line version.

In [15]:
from __future__ import absolute_import, division, print_function

from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

2.0.0-alpha0


In [16]:
file_path = Path('../tsv')

In [17]:
file_name = 'SN0204288721.tsv'
file_path = file_path.joinpath(file_name)

In [18]:
file_path

PosixPath('../tsv/SN0204288721.tsv')

# Using tf.data.TextLineDataset with skip operation

In [112]:
# skip the comments but not the headers
# the comments are at lines 1-31
# the column headings is at line 32
# the 5s 15s 20d 14n ... line ends is at 33
num_lines_to_skip = 33
text_lines_ds = tf.data.TextLineDataset(str(file_path)).skip(num_lines_to_skip)

In [113]:
tf.print(text_lines_ds)

<SkipDataset shapes: (), types: tf.string>


In [114]:
# Just print 5 lines including the header
limit = 5
for i in text_lines_ds:
    limit = limit - 1
    if (limit < 0):
        break
    tf.print(i)

USGS	0204288721	2016-04-21	1.70	A	-1.48	A	0.00	A
USGS	0204288721	2016-04-22	1.82	A	-1.56	A	0.38	A
USGS	0204288721	2016-04-23	2.24	A	-1.37	A	0.08	A
USGS	0204288721	2016-04-24	2.06	A	-1.03	A	0.00	A
USGS	0204288721	2016-04-25	2.02	A	-1.19	A	0.01	A


In [115]:
# this looks like it read each line as one string

# with tf.data.experimental.CsvDataset  with deliminator set to \t

In [107]:
num_lines_to_skip = 34

In [134]:
text_lines_ds = tf.data.experimental.CsvDataset(filenames=str(file_path),
                                                record_defaults = [tf.string,  # usgs
                                                                   tf.int64,   # site_no
                                                                   tf.string,  # date_time
                                                                   tf.float32,  # Tidal High
                                                                   tf.string,  # Tidal High qual code
                                                                   tf.float32,  # Tidal Low
                                                                   tf.string,  # Tidal Low qual code
                                                                   tf.float32,  # Precipitation (inches)
                                                                   tf.string], # Precip qual code
                                                header=False,
                                                field_delim='\t',
                                                select_cols=[1,2]
                                               ).skip(num_lines_to_skip)

InvalidArgumentError: select_cols should match output size [Op:ExperimentalCSVDataset]

In [130]:
text_lines_ds

<SkipDataset shapes: ((), (), (), (), (), (), (), (), ()), types: (tf.string, tf.int64, tf.string, tf.float32, tf.string, tf.float32, tf.string, tf.float32, tf.string)>

In [131]:
tf.print(text_lines_ds)

<SkipDataset shapes: ((), (), (), (), (), (), (), (), ()), types: (tf.string, tf.int64, tf.string, tf.float32, tf.string, tf.float32, tf.string, tf.float32, tf.string)>


In [132]:
for element in text_lines_ds:
    print(element)

InvalidArgumentError: Expect 9 fields but have 1 in record [Op:IteratorGetNextSync]

In [91]:
# Just print 5 lines including the header
limit = 5
for i in text_lines_ds:
    limit = limit - 1
    if (limit < 0):
        break
    tf.print(i)

InvalidArgumentError: Expect 9 fields but have 1 in record [Op:IteratorGetNextSync]

# Try to use a csv file after converting with bash and sed

In [135]:
file_path = Path('../csv')
file_name = 'SN0204288721.csv'
file_path = file_path.joinpath(file_name)

In [136]:
num_lines_to_skip = 0

In [172]:
text_lines_ds = tf.data.experimental.CsvDataset(filenames=str(file_path),
                                                record_defaults = [tf.string,   # usgs SKIP
                                                                   tf.int64,    # site_no
                                                                   tf.string,   # date_time
                                                                   tf.float32,  # Tidal High
                                                                   tf.string,   # Tidal High qual code  SKIP
                                                                   tf.float32,  # Tidal Low
                                                                   tf.string,   # Tidal Low qual code  SKIP
                                                                   tf.float32,  # Precipitation (inches)
                                                                   tf.string
                                                ],  # Precip qual code SKIP
                                                header=True
                                               ).skip(num_lines_to_skip)

In [173]:
tf.print(text_lines_ds)

<SkipDataset shapes: ((), (), (), (), (), (), (), (), ()), types: (tf.string, tf.int64, tf.string, tf.float32, tf.string, tf.float32, tf.string, tf.float32, tf.string)>


In [174]:
# Just print 5 lines including the header
limit = 5
for i in text_lines_ds:
    limit = limit - 1
    if (limit < 0):
        break
    tf.print(i)

("USGS", 204288721, "2016-04-21", 1.7, "A", -1.48, "A", 0, "A")
("USGS", 204288721, "2016-04-22", 1.82, "A", -1.56, "A", 0.38, "A")
("USGS", 204288721, "2016-04-23", 2.24, "A", -1.37, "A", 0.08, "A")
("USGS", 204288721, "2016-04-24", 2.06, "A", -1.03, "A", 0, "A")
("USGS", 204288721, "2016-04-25", 2.02, "A", -1.19, "A", 0.01, "A")


## Using the select_cols parameter breaks it

In [175]:
text_lines_ds = tf.data.experimental.CsvDataset(filenames=str(file_path),
                                                record_defaults = [tf.string,   # usgs SKIP
                                                                   tf.int64,    # site_no
                                                                   tf.string,   # date_time
                                                                   tf.float32,  # Tidal High
                                                                   tf.string,   # Tidal High qual code  SKIP
                                                                   tf.float32,  # Tidal Low
                                                                   tf.string,   # Tidal Low qual code  SKIP
                                                                   tf.float32,  # Precipitation (inches)
                                                                   tf.string],  # Precip qual code SKIP
                                                header=True,
                                                select_cols=[2,3,4,6,8]
                                               ).skip(num_lines_to_skip)

InvalidArgumentError: select_cols should match output size [Op:ExperimentalCSVDataset]

## How about if we skip specifying col default values?
It does work.  Cols start counting at zero.

In [190]:
text_lines_ds = tf.data.experimental.CsvDataset(filenames=str(file_path),
                                                record_defaults = [tf.int64,    # site_no
                                                                   tf.string,  # date_time
                                                                   tf.float32,  # Tidal High
                                                                   tf.float32,  # Tidal Low
                                                                   tf.float32], # Precipitation (inches)
                                                                   
                                                header=True,
                                                select_cols=[1,2,3,5,7]
                                               )

In [191]:
# Just print 5 lines including the header
limit = 5
for i in text_lines_ds:
    limit = limit - 1
    if (limit < 0):
        break
    tf.print(i)

(204288721, "2016-04-21", 1.7, -1.48, 0)
(204288721, "2016-04-22", 1.82, -1.56, 0.38)
(204288721, "2016-04-23", 2.24, -1.37, 0.08)
(204288721, "2016-04-24", 2.06, -1.03, 0)
(204288721, "2016-04-25", 2.02, -1.19, 0.01)


In [194]:
text_lines_ds

<CsvDatasetV2 shapes: ((), (), (), (), ()), types: (tf.int64, tf.string, tf.float32, tf.float32, tf.float32)>